### Removing Duplicates

In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.DataFrame({'k1': ['one', 'two'] * 3 + ['two'],
                     'k2': [1, 1, 2, 3, 3, 4, 4]})
data

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


In [3]:
# whether each row is duplicated(has been observed in a previous row) or not
data.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

In [4]:
# drop_duplicates returns a DataFrame where the duplicated
data.drop_duplicates()

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


In [5]:
# both of these methods by default consider all of the columns
# we can specify particular columns
data['v1'] = range(7)
data

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
5,two,4,5
6,two,4,6


In [6]:
data.drop_duplicates(['k1'])

,k1,k2,v1
0,one,1,0
1,two,1,1


In [7]:
# duplicated and drop_duplicates by default keep the first observed value 
# passing keep='last' will return the last one
data.drop_duplicates(['k1','k2'], keep = 'last')

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
6,two,4,6


### Transforming Data Using a Function or Mapping

In [8]:
data = pd.DataFrame({'food': ['bacon', 'pulled pork', 'bacon',
   ....:                               'Pastrami', 'corned beef', 'Bacon',
   ....:                               'pastrami', 'honey ham', 'nova lox'],
   ....:                      'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,Pastrami,6.0
4,corned beef,7.5
5,Bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


In [9]:
# add a column indicating the type of animal that each food came from
# let's write down a mapping of each distinct meat type to the kind of animal
meat_to_animal = {
  'bacon': 'pig',
  'pulled pork': 'pig',
  'pastrami': 'cow',
  'corned beef': 'cow',
  'honey ham': 'pig',
  'nova lox': 'salmon'
}

In [10]:
lowercased = data['food'].str.lower()
lowercased

0          bacon
1    pulled pork
2          bacon
3       pastrami
4    corned beef
5          bacon
6       pastrami
7      honey ham
8       nova lox
Name: food, dtype: object

In [11]:
data['animal'] = lowercased.map(meat_to_animal)
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


In [12]:
# we could also passed a function that does all the work:
data['food'].map(lambda x:meat_to_animal[x.lower()])

0       pig
1       pig
2       pig
3       cow
4       cow
5       pig
6       cow
7       pig
8    salmon
Name: food, dtype: object

## Replacing Values

In [13]:
data = pd.Series([1., -999., 2., -999., -1000., 3.])
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

In [14]:
# the -999 values might be sentinel values for missing data.
# To replace these with NA values that pandas understands, we can use replace
data.replace(-999, np.nan)

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

In [15]:
data.replace([-999,-1000], np.nan)

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

In [16]:
data.replace([-999,-1000], [np.nan,0])

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

In [17]:
# the argument passed can also be a dict
data.replace({-999:np.nan, -1000:0})

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

### Renaming Axis Indexes

In [18]:
# axis labels can be similarly transformed by a function or mapping
data = pd.DataFrame(np.arange(12).reshape((3, 4)),
                     index=['Ohio', 'Colorado', 'New York'],
                     columns=['one', 'two', 'three', 'four'])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


In [19]:
transform = lambda x:x[:4].upper()
data.index.map(transform)

Index(['OHIO', 'COLO', 'NEW '], dtype='object')

In [20]:
# you can assign to index, modifying the DataFrame in-place
data.index = data.index.map(transform)
data

,one,two,three,four
OHIO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


In [21]:
# If you want to create a transformed version of a dataset
data.rename(index=str.title,columns=str.upper)

,ONE,TWO,THREE,FOUR
Ohio,0,1,2,3
Colo,4,5,6,7
New,8,9,10,11


In [22]:
# inplace
data.rename(index={'OHIO':'INDIANA'}, inplace=True)
data

,one,two,three,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


### Discretization and Binning

In [23]:
# Continuous data is often discretized or otherwise separated into “bins” for analysis
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]
bins = [18,25,35,60,100]
cats = pd.cut(ages, bins)
cats

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, interval[int64]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

In [24]:
cats.codes

array([0, 0, 0, 1, 0, 0, 2, 1, 3, 2, 2, 1], dtype=int8)

In [25]:
cats.categories

IntervalIndex([(18, 25], (25, 35], (35, 60], (60, 100]],
              closed='right',
              dtype='interval[int64]')

In [26]:
pd.value_counts(cats)

(18, 25]     5
(35, 60]     3
(25, 35]     3
(60, 100]    1
dtype: int64

In [27]:
# change which side is closed by passing right=False
pd.cut(ages,[18,26,36,61,100],right=False)

[[18, 26), [18, 26), [18, 26), [26, 36), [18, 26), ..., [26, 36), [61, 100), [36, 61), [36, 61), [26, 36)]
Length: 12
Categories (4, interval[int64]): [[18, 26) < [26, 36) < [36, 61) < [61, 100)]

In [28]:
# you can also pass your won bin names
group_names = ['Youth', 'YoungAdult', 'MiddleAged', 'Senior']
pd.cut(ages, bins, labels = group_names)

[Youth, Youth, Youth, YoungAdult, Youth, ..., YoungAdult, Senior, MiddleAged, MiddleAged, YoungAdult]
Length: 12
Categories (4, object): [Youth < YoungAdult < MiddleAged < Senior]

In [29]:
# if you pass a integer number of bins to cut instead of explicit bin edges
# it will compute equal-length bins based on the minimum and maximum values
data = np.random.rand(20)
pd.cut(data, 4, precision=2)

[(0.013, 0.26], (0.013, 0.26], (0.74, 0.99], (0.74, 0.99], (0.74, 0.99], ..., (0.26, 0.5], (0.74, 0.99], (0.26, 0.5], (0.26, 0.5], (0.013, 0.26]]
Length: 20
Categories (4, interval[float64]): [(0.013, 0.26] < (0.26, 0.5] < (0.5, 0.74] < (0.74, 0.99]]

In [30]:
data = np.random.randn(1000)  # Normally distributed
cats = pd.qcut(data,4) #cut into quartiles
cats

[(-0.662, -0.0131], (-0.0131, 0.676], (-3.072, -0.662], (-0.0131, 0.676], (-0.0131, 0.676], ..., (-3.072, -0.662], (-0.0131, 0.676], (-3.072, -0.662], (-0.0131, 0.676], (-3.072, -0.662]]
Length: 1000
Categories (4, interval[float64]): [(-3.072, -0.662] < (-0.662, -0.0131] < (-0.0131, 0.676] < (0.676, 3.199]]

In [31]:
pd.value_counts(cats)

(0.676, 3.199]       250
(-0.0131, 0.676]     250
(-0.662, -0.0131]    250
(-3.072, -0.662]     250
dtype: int64

In [32]:
# you can pass your own quantiles(numbers between 0 and 1)
cats = pd.qcut(data, [0, 0.1, 0.5, 0.9, 1.])
cats

[(-1.25, -0.0131], (-0.0131, 1.187], (-3.072, -1.25], (-0.0131, 1.187], (-0.0131, 1.187], ..., (-1.25, -0.0131], (-0.0131, 1.187], (-3.072, -1.25], (-0.0131, 1.187], (-1.25, -0.0131]]
Length: 1000
Categories (4, interval[float64]): [(-3.072, -1.25] < (-1.25, -0.0131] < (-0.0131, 1.187] < (1.187, 3.199]]

In [33]:
cats.value_counts()

(-3.072, -1.25]     100
(-1.25, -0.0131]    400
(-0.0131, 1.187]    400
(1.187, 3.199]      100
dtype: int64

### Detecting and Filtering Outliers

In [34]:
data = pd.DataFrame(np.random.randn(1000, 4)) # normally distributed data
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.026046,0.004753,-0.027591,-0.007522
std,0.991195,1.001706,0.977580,1.028220
min,-3.098960,-3.004077,-2.883640,-3.452886
25%,-0.648510,-0.708783,-0.661272,-0.744463
50%,-0.013235,0.002517,-0.065511,-0.004272
75%,0.667190,0.668056,0.621563,0.685479
max,3.569390,3.584455,2.956771,3.436065


In [35]:
# suppose you want to find values in one of the columns exceeding 3 in absolute value
col = data[2]
col[np.abs(col)>3]

Series([], Name: 2, dtype: float64)

In [36]:
# To select all rows having a value exceeding 3 or -3, you can use any method
data[(np.abs(data)>3).any(1)]

,0,1,2,3
42,0.345290,3.584455,-2.883640,0.807544
147,0.646496,-3.004077,0.194927,-0.335080
234,0.056537,0.618097,-0.420237,3.162646
257,3.002470,0.043296,1.522996,0.106172
380,-2.042241,-1.100951,-0.222598,3.182377
389,-2.155061,3.256501,-0.324780,-0.158441
404,-3.098960,0.246178,-0.595546,1.402292
458,-2.144304,-1.679138,-0.839324,-3.094146
617,0.674361,0.344529,0.951986,-3.024854
655,3.569390,0.494072,0.063705,-0.205159


In [37]:
(np.abs(data)>3).any(0)

0     True
1     True
2    False
3     True
dtype: bool

In [38]:
(np.abs(data)>3).any(1)[:5]

0    False
1    False
2    False
3    False
4    False
dtype: bool

In [39]:
# cap values outside the interval -3 to 3
# e.g. -4 and 4 will set to -3 and 3
data[np.abs(data)>3] = np.sign(data)*3
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.025291,0.003282,-0.027591,-0.007908
std,0.988117,0.996948,0.977580,1.023497
min,-3.000000,-3.000000,-2.883640,-3.000000
25%,-0.648510,-0.708783,-0.661272,-0.744463
50%,-0.013235,0.002517,-0.065511,-0.004272
75%,0.667190,0.668056,0.621563,0.685479
max,3.000000,3.000000,2.956771,3.000000


In [40]:
np.sign(data).head()

,0,1,2,3
0,1.0,1.0,1.0,1.0
1,1.0,1.0,1.0,-1.0
2,1.0,1.0,1.0,-1.0
3,1.0,1.0,-1.0,1.0
4,-1.0,-1.0,-1.0,1.0


### Permutation and Random Sampling(randomly reordering)

In [41]:
df = pd.DataFrame(np.arange(5 * 4).reshape((5, 4)))
df

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19


In [42]:
sampler = np.random.permutation(5)
sampler

array([1, 0, 2, 3, 4])

In [43]:
df.take(sampler)

,0,1,2,3
1,4,5,6,7
0,0,1,2,3
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19


In [44]:
df.loc[sampler.tolist(),:]

,0,1,2,3
1,4,5,6,7
0,0,1,2,3
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19


In [45]:
df.iloc[sampler.tolist()]

,0,1,2,3
1,4,5,6,7
0,0,1,2,3
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19


In [46]:
pd.DataFrame(df, index=sampler.tolist())

,0,1,2,3
1,4,5,6,7
0,0,1,2,3
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19


In [47]:
# To select a random subset without replacement, you can use the sample method
df.sample(n=3)

,0,1,2,3
1,4,5,6,7
4,16,17,18,19
3,12,13,14,15


In [48]:
# To generate a sample with replacement, pass replace=True
choices = pd.Series([5,7,-1,6,4])
draws = choices.sample(n=10, replace=True)
draws

0    5
1    7
2   -1
3    6
0    5
1    7
4    4
1    7
1    7
2   -1
dtype: int64

## Computing Indicator/Dummy Variables

In [49]:
df = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
   .....:                    'data1': range(6)})
df

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [50]:
# converting a categorical variable into a dummy or indicator matrix
pd.get_dummies(df['key'])

,a,b,c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


In [51]:
dummies = pd.get_dummies(df['key'], prefix='key')
dummies

,key_a,key_b,key_c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


In [52]:
df_with_dummy = df[['data1']].join(dummies)
df_with_dummy

,data1,key_a,key_b,key_c
0,0,0,1,0
1,1,0,1,0
2,2,1,0,0
3,3,0,0,1
4,4,1,0,0
5,5,0,1,0


In [53]:
mnames = ['movie_id', 'title', 'genres']
movies = pd.read_table('./datasets/movies.dat', sep='::',
                        header=None, names=mnames)
movies[:10]

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: read_table is deprecated, use read_csv instead.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  This is separate from the ipykernel package so we can avoid doing imports until


,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [54]:
all_genres = []
for x in movies.genres:
    all_genres.extend(x.split('|'))
genres = pd.unique(all_genres)
genres

array(['Animation', "Children's", 'Comedy', 'Adventure', 'Fantasy',
       'Romance', 'Drama', 'Action', 'Crime', 'Thriller', 'Horror',
       'Sci-Fi', 'Documentary', 'War', 'Musical', 'Mystery', 'Film-Noir',
       'Western'], dtype=object)

In [55]:
# one way to construct indicator DataFrame is to start with a DataFrame of all zeros
zero_matrix = np.zeros((len(movies),len(genres)))
dummies = pd.DataFrame(zero_matrix, columns = genres)

In [56]:
gen = movies.genres[0]
gen.split('|')

['Animation', "Children's", 'Comedy']

In [57]:
dummies.columns.get_indexer(gen.split('|'))

array([0, 1, 2], dtype=int64)

In [58]:
# Then we can use .iloc to set values based on these indices:
for i,gen in enumerate(movies.genres):
    indices = dummies.columns.get_indexer(gen.split('|'))
    dummies.iloc[i,indices] = 1
movies_windic = movies.join(dummies.add_prefix('Genre_'))
movies_windic.iloc[0]

movie_id                                       1
title                           Toy Story (1995)
genres               Animation|Children's|Comedy
Genre_Animation                                1
Genre_Children's                               1
Genre_Comedy                                   1
Genre_Adventure                                0
Genre_Fantasy                                  0
Genre_Romance                                  0
Genre_Drama                                    0
Genre_Action                                   0
Genre_Crime                                    0
Genre_Thriller                                 0
Genre_Horror                                   0
Genre_Sci-Fi                                   0
Genre_Documentary                              0
Genre_War                                      0
Genre_Musical                                  0
Genre_Mystery                                  0
Genre_Film-Noir                                0
Genre_Western       

In [59]:
# 
np.random.seed(12345)
values = np.random.rand(10)
values

array([0.92961609, 0.31637555, 0.18391881, 0.20456028, 0.56772503,
       0.5955447 , 0.96451452, 0.6531771 , 0.74890664, 0.65356987])

In [60]:
bins = [0,0.2,0.4,0.6,0.8,1]
pd.get_dummies(pd.cut(values, bins))

   (0.0, 0.2]  (0.2, 0.4]  (0.4, 0.6]  (0.6, 0.8]  (0.8, 1.0]
0           0           0           0           0           1
1           0           1           0           0           0
2           1           0           0           0           0
3           0           1           0           0           0
4           0           0           1           0           0
5           0           0           1           0           0
6           0           0           0           0           1
7           0           0           0           1           0
8           0           0           0           1           0
9           0           0           0           1           0